# Part-II RGB Camera Calibration Using SVD

>## Introduction
This program calibrates an RGB camera using the SVD routine.
The input files are a file, containing a list of 3D coordinates, and a file, containing the corresponding 2D pixel coordinates.
The output is a 3x4 calibration matrix and the average errors (difference between projected points and original pixels).


In [ ]:
import numpy as np
from scipy import linalg as la

>## 1 Read Input Files and Construct Matrix $\textbf{A}$
In this section, we read $n$ pairs of points, $(P_i, p_i)$, from input files containing 2D and 3D coordinates. \\
Then, we construct the matrix $\textbf{A}\in \textbf{M}_{(2n)\times 12}(\mathbb{R})$ as follows,
\begin{align}
        \textbf{A}
        &= \begin{pmatrix}
        \textbf{G_1} \\
        \textbf{G_2} \\
        ... \\
        \textbf{G_n}
    \end{pmatrix}, \text{where } \textbf{G}_j
        =\begin{pmatrix}
        -X_j & -Y_j & -Z_j & -1 & \vec{0}^\top & u_jX_j & u_jY_j & u_jZ_j & u_j \\
        \vec{0}^\top & -X_j & -Y_j & -Z_j & -1 & v_jX_j & v_jY_j & v_jZ_j & v_j
    \end{pmatrix} \\
        &=\begin{pmatrix}
        -X_1 & -Y_1 & -Z_1 & -1 & 0 & 0 & 0 & 0 & u_1X_1 & u_1Y_1 & u_1Z_1 & u_1 \\
        0 & 0 & 0 & 0 & -X_1 & -Y_1 & -Z_1 & -1 & v_1X_1 & v_1Y_1 & v_1Z_1 & v_1 \\
        -X_2 & -Y_2 & -Z_2 & -1 & 0 & 0 & 0 & 0 & u_2X_2 & u_2Y_2 & u_2Z_2 & u_2 \\
        0 & 0 & 0 & 0 & -X_2 & -Y_2 & -Z_2 & -1 & v_2X_2 & v_2Y_2 & v_2Z_2 & v_2 \\
        &&&&&... \\
        -X_n & -Y_n & -Z_n & -1 & 0 & 0 & 0 & 0 & u_nX_n & u_nY_n & u_nZ_n & u_n \\
        0 & 0 & 0 & 0 & -X_n & -Y_n & -Z_n & -1 & v_nX_n & v_nY_n & v_nZ_n & v_n
        \end{pmatrix}
    \end{align}
In addition, we initialize the matrix $P$ from the input 3D coordinate file, defined as the following,
\begin{align} P = \begin{pmatrix}
        X_1 & ... & X_i & ... & X_n \\
        Y_1 & ... & Y_i & ... & Y_n \\
        Z_1 & ... & Z_i & ... & Z_n \\
        1   & ... & 1   & ... & 1
    \end{pmatrix} \in M_{4\times n}(\mathbb{R})
    \end{align}
Also, we initialize the matrix $Q$ from the input 2D coordinate file, defined as the following,
\begin{align} Q = \begin{pmatrix}
        u_1 & ... & u_i & ... & u_n \\
        v_1 & ... & v_i & ... & v_n \\
        1   & ... & 1   & ... & 1
    \end{pmatrix} \in M_{3\times n}(\mathbb{R})
    \end{align}
Based on the perspective projection equation, $P$ and $Q$ are expected to have the following property,
\begin{align} \exists \lambda_1, ..., \lambda_n,
        Q = MP(\lambda_1, ..., \lambda_n)^\top
    \end{align}

In [ ]:
# define paths of the two input files
path_2d = "/content/2D.txt"
path_3d = "/content/3D.txt"

# opening both the files in reading modes
with open(path_2d) as f_2d, open(path_3d) as f_3d:
    num_line_2d = int(f_2d.readline().split('\n')[0])
    num_line_3d = int(f_3d.readline().split('\n')[0])
    if num_line_2d != num_line_3d:
        print("Error: Number of Points does NOT Match in 2D.text and 3D.txt")
        exit()
    # initialize matrics P and Q for input files
    P = np.ones((4, num_line_3d))
    Q = np.ones((3, num_line_3d))

    # initialize Matrix A
    A = np.zeros((num_line_3d * 2, 12))

    # fill Matrix P, Q, and A
    while num_line_2d > 0:
        # define the index of point, i
        i = num_line_3d - num_line_2d

        # define 2d coord (u,v)
        line_2d = f_2d.readline().split(' ')
        u = float(line_2d[0])
        v = float(line_2d[1])

        # fill matrix Q with u, v
        Q[0, i] = u
        Q[1, i] = v

        # define 3d coord (x,y,z)
        line_3d_raw = f_3d.readline().split(' ')
        line_3d_fl = []
        for j in range(len(line_3d_raw)):
            if len(line_3d_raw[j]) > 0:
                line_3d_fl.append(float(line_3d_raw[j]))
        x = line_3d_fl[0]
        y = line_3d_fl[1]
        z = line_3d_fl[2]

        # fill matrix P with x, y, z
        P[0, i] = x
        P[1, i] = y
        P[2, i] = z

        # fill rows 2i and 2i+1 in A
        A[i * 2, 0] = -x
        A[i * 2, 1] = -y
        A[i * 2, 2] = -z
        A[i * 2, 3] = -1
        A[i * 2, 8] = u * x
        A[i * 2, 9] = u * y
        A[i * 2, 10] = u * z
        A[i * 2, 11] = u
        A[i * 2 + 1, 4] = -x
        A[i * 2 + 1, 5] = -y
        A[i * 2 + 1, 6] = -z
        A[i * 2 + 1, 7] = -1
        A[i * 2 + 1, 8] = v * x
        A[i * 2 + 1, 9] = v * y
        A[i * 2 + 1, 10] = v * z
        A[i * 2 + 1, 11] = v

        # update
        num_line_2d -= 1

#print(P)
#print(Q)

>## 2 Find Eigenvalues and Eigenvectors of Matrix $\textbf{A}$ Using SVD Routines
In this section, we will find $V=[v_1,...,v_n]$ and $\sigma_1,...,\sigma_p$ such that
\begin{align}
        & A=USV^\top, \text{where } S=diag(\sigma_1,...,\sigma_p) \\
        \text{and } & Av_i = \sigma_i u_i, \forall i\leq p
    \end{align}
Then, we extract the smallest eigenvalue and the corresponding eigenvector. The chosen eigenvector ($\vec{w}$) contains all the values in our calibration matrix ($M$) denoted as follows,
\begin{align}
        &\vec{w} = \begin{pmatrix}
        m_{11} \\
        m_{12} \\
        m_{13} \\
        m_{14} \\
        ... \\
        m_{31} \\
        m_{32} \\
        m_{33} \\
        m_{34} \\
    \end{pmatrix} \\
    \text{and } & M = \begin{pmatrix}
        m_{11}, m_{12}, m_{13}, m_{14} \\
        m_{21}, m_{22}, m_{23}, m_{24} \\
        m_{31}, m_{32}, m_{33}, m_{34} \\
    \end{pmatrix}
    \end{align}

In [ ]:
# apply the SVD routine s.t. A=m_u m_s m_vh
m_u, m_s, m_vh = la.svd(A)

# find the smallest eigenvalue
eigen_index = 0
eigenval_s = m_s[0]
for i in range(m_s.shape[0]):
    if m_s[i] < eigenval_s:
        eigen_index = i
        eigenval_s = m_s[i]

# find the corresponding eigenvector
eigenvec_s = m_vh[eigen_index]

# check the correctness based on Av=\sigma u
left_side = np.matmul(A, eigenvec_s)
right_side = eigenval_s * m_u[:, eigen_index]
print("The eigenvector is valid because the error = ", la.norm(left_side - right_side),
      "is significantly small")

# define the matrix M
mat_m = eigenvec_s.reshape((3,4))
print("The calibration matrix M is\n", mat_m)



The eigenvector is valid because the error =  1.0263514835565073e-12 is significantly small
The calibration matrix M is
 [[-2.75988290e-02 -8.24928775e-16 -7.06531569e-03 -7.06529955e-01]
 [ 1.35499928e-14 -2.75988290e-02 -7.06531569e-03 -7.06529955e-01]
 [ 4.04813986e-17  6.46182950e-17 -2.75988894e-05 -2.75988264e-03]]


>## 3 Evaluation Based on the Average Error
In this section, we will produce the average error between projected points and original pixels. \\
Let the original pixels be stored in the matrix $Q$ such that
\begin{align} Q &= \begin{pmatrix}
        \vec{q_1} & ... & \vec{q_i} & ... & \vec{q_n} \\
        1   & ... & 1   & ... & 1
    \end{pmatrix} \\
        &= \begin{pmatrix}
        u_1 & ... & u_i & ... & u_n \\
        v_1 & ... & v_i & ... & v_n \\
        1   & ... & 1   & ... & 1
    \end{pmatrix}
    \end{align}
Let $Q'$ be matrix that stores our projected points calculated by the calibration matrix $M$ and the 3D coordinate matrix $P$, satisfying $Q' = MP(\lambda_1, ..., \lambda_n)^\top$, and denote $Q'$ as follows,
\begin{align} Q' &= \begin{pmatrix}
        \vec{q_1'} & ... & \vec{q_i'} & ... & \vec{q_n'} \\
        1   & ... & 1   & ... & 1
    \end{pmatrix}
    \end{align} \\
Now, the error between two corresponding points is defined by
\begin{align}
        \epsilon_i = \lvert\lvert \vec{q_i} - \vec{q_i'} \rvert\rvert
    \end{align}
and the average error $E_{avg}$ is
\begin{align}
        E_{avg} = \frac{\sum_{i=1}^{n}\epsilon_i}{n}
    \end{align}

In [ ]:
# compute Q'
Q_prime = np.matmul(mat_m, P)
for j in range(num_line_3d):
    lamb = Q_prime[2, j]
    if lamb == 0:
        continue
    Q_prime[0, j] /= lamb
    Q_prime[1, j] /= lamb
    Q_prime[2, j] /= lamb

# compute E_total
n = num_line_3d
err_total = 0
for i in range(n):
    q_i = Q[:-1, i]
    q_i_prime = Q_prime[:-1, i]
    err_i = la.norm(q_i - q_i_prime)
    err_total += err_i

# compute E_avg
err_avg = err_total / n
print("The aveage error is ", err_avg)

The aveage error is  1.3756196385764567e-05


>## 4 Conclusion
The calibration matrix we calculated in section 2 is valid because the average error is significantly small.